# Extract SNPs from gentoype data


## Overview

This notebook allows the extraction of individual SNPs from the UK Biobank genotyping data. 
<br>
SNPs of interest can specified in the following ways:

* A list of individual SNPs
* Genomic coordinates

* A text file containing an rsID for each SNP on a separate line, for example:

rs683743847
<br>
rs73473437

* A text file containing genomic coordinates, where the first column lists the chromosome, the following two columns specify the start and end coordinates and the last column provides an identifier for the region. Do not include column headers.

    |Column1| Column2| Column3| Column4|
    |-------|--------|--------|--------|
    |2 | 30000000 | 35000000 | R| 
    |2 | 60000000 | 62000000 | R2
    |X |10000000  | 20000000 | R3
    
     
     



In order to extract SNPs, the plink binary files holding the genotype calls are downloaded to the instance and merged into a single binary file. This file is then used for SNP extraction. Although these operations could be performed for all files containing genotype calls, it is faster and less computationally intensive to specify chromosomes of interest first and download and merge only genotype call files relevant to the chromosomes of interest. 



# How to run this notebook

Follow the steps below to run this Jupiter Notebook:

* Click on the Tools menu and select "JupiterLab"
* Click on the "New JupiterLab" button to start a JupiterLab instance.
* Select a name and a project from the dropdown meny for your JupiterLab environment.
* Select the priority for you JupiterLab environment; "Normal" of "High" is recommended.
* Under "Cluster Configuration", select Single Node.
* Set instance type and duration for you environment.If the chromosomes where SNPs of interest are known in advance and a small number of chromosomes files are  selected for download, an instance type with such as mem1_hdd1_v2_x8 or mem1_hdd1_v2_x4 should be sufficient. If all chromosome files are selected for download, instance type with more computational resources may be needed (mem1_hdd1_v2_x16 or above).
* Click on "Start Environment"
* You will see your environment go from "Initialising" to "Launching" and then "Ready". This may take some time depending on the priority selected; at busy times, it may be necessary to select high priority to avoid long initialising times. Once the environment is ready, click on "Open".
* A JuiterLab session will open. On the left side of the screen, you will see a a "DNA Nexus" tab, allowing you to open notebooks directly from your project environment. If you have saved this notebook under you project environment, just double click to open it.
* Press "Ctrl" + "Enter" to run code cells. An hourglass icon on the JupireLab tab in your browser indicates that the code is running. Please note that depending on number of chromosomes and SNPs and your instance type, code may take some time to run.

## Install plink 

Plink is installed using conda and plink --help is called to verify the installation.

In [1]:
%%bash

# install plink
conda install -c bioconda plink
plink --help | head

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - plink


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.4.26  |       h06a4308_0         124 KB
    plink-1.90b6.21            |       h516909a_0         6.8 MB  bioconda
    ------------------------------------------------------------
                                           Total:         7.0 MB

The following NEW packages will be INSTALLED:

  plink              bioconda/linux-64::plink-1.90b6.21-h516909a_0

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> pkgs/main::ca-certificates-2022.4.26-h06a4308_0


Proceed ([y]/n)? Invalid choice: plink --help | head
Proceed ([y]/n)? 

ca-certificates-2022 | 124 KB    | ########## | 100% 
plink-1.90b6.21      | 6.8 MB    | ########## | 100% 

## Download genotyping files 

Plink files containing genotypes are downloaded to the local instance using the dx download command.

If the chromosome location of the SNPs is known in advance , it may be best to limit the number of downloaded files to files for the chromosomes needed using the options below.


In [2]:
%%bash

# Use the code below to download files for chromosomes of interest


# list all available genotype calls files and places them in a file
# The file all_snp_files.txt will appear in your environment (click on the folder icon on the left of your screen to view files in your environment)
dx find data --name "ukb22418_*.bed" --path "/Bulk/Genotype Results/Genotype calls/" --delim ',' | awk -F, '{ print $4 }' |awk -F'/' '{print $5}' | awk -F'.' '{print $1}'  > all_snp_files.txt


# The code below provides an example on how to download data for  chromosome 1, chromosome 4 and chromosome X
# To modify, replace  the 1,4, and X in the followig expression '^ukb22418_c[14X]_' with your chromosome of interest
# For example, if you are interested in SNPs on chromosomes 5,6,8, and 9, use: '^ukb22418_c[5689]_'
# For double-digit chromosomes , e.g chromosome 12 use the following format: '^ukb22418_c[1][2]_'
# For mixed double digit and single digit chromosomes, e,g, chromosome 8 and 12, use the following format: '^ukb22418_c[18][2]_'
# To include all chromosomes modify the expression to '^ukb22418_c*_' 



files=$(grep '^ukb22418_c[14X]_' all_snp_files.txt) 
echo $files 




# Download files containing chromosomes of interest

for i in $files; do
dx download "/Bulk/Genotype Results/Genotype calls/${i}*"
done



# The downloaded files will appear in your environment. There should be a bed, bim and fam file for each chromosome containing SNPs of interest.



ukb22418_cX_b0_v2 ukb22418_c4_b0_v2 ukb22418_c1_b0_v2


## Merge genotyping files

Genotyping files containing data for chromosomes of interest are merged into a single binary format file, genotyping_merged.
The following files should appear in the file list:

genotyping_merged.bed
<br>
genotyping_merged.bim
<br>
genotyping_merged.fam

Please note code in this cell might take some time to run.

In [3]:
%%bash

# Define files to merge. Please replace the '^ukb22418_c[14X]_' expression in the code below with the expression used to define chromosomes in the previous code chunk. 

grep '^ukb22418_c[14X]_' all_snp_files.txt > files_to_merge.txt
cat files_to_merge.txt
plink --merge-list files_to_merge.txt --make-bed --out genotyping_merged

ukb22418_cX_b0_v2
ukb22418_c4_b0_v2
ukb22418_c1_b0_v2
PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to genotyping_merged.log.
Options in effect:
  --make-bed
  --merge-list files_to_merge.txt
  --out genotyping_merged

15544 MB RAM detected; reserving 7772 MB for main workspace.
Performing 3-pass merge (488377 people, 44408/129787 variants per pass).
Pass 1 complete.                              
Pass 2 complete.                              
Merged fileset written to genotyping_merged-merge.bed +
genotyping_merged-merge.bim + genotyping_merged-merge.fam .
129787 variants loaded from .bim file.
488377 people (223429 males, 264742 females, 206 ambiguous) loaded from .fam.
Ambiguous sex IDs written to genotyping_merged.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 488377 founders and 0 nonfounders present.
Calculating allel

## Extarcting individual SNP IDs

Use the code cell below if you want to extract data for a small number of SNPs.
Provide a list of SNP rsIDs, separated by commas.
Your results can be found in the snp_ind_plink_results.raw The file is uploaded to your project space and can be downloaded from there.


In [4]:
%%bash
# replace the rsIDs in the code below with your SNPs; Always seperate each rsID with a commma.
plink --bfile genotyping_merged --snps rs28659788,rs116587930 --recode A --out snp_ind_plink_results
dx upload snp_ind_plink_results.raw

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to snp_ind_plink_results.log.
Options in effect:
  --bfile genotyping_merged
  --out snp_ind_plink_results
  --recode A
  --snps rs28659788,rs116587930

15544 MB RAM detected; reserving 7772 MB for main workspace.
129787 variants loaded from .bim file.
488377 people (223429 males, 264742 females, 206 ambiguous) loaded from .fam.
Ambiguous sex IDs written to snp_ind_plink_results.nosex .
--snps: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 488377 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.500514.
2 variants and 488377 people pa

## Extracting a list of SNPs provided in a text file

Use the code cell below if you have a file listsing rsIDs of interest.
Prepare a file containing rsIDs (one per line). Rename it to snps_list.txt and upload to your project folder.
The file is downloaded to the instance using the dx dowload command and used to extract SNPs. 
Your results can be found in the snp_list_plink_results.raw The file is uploaded to your project space and can be downloaded from there.


In [5]:
%%bash

dx download -f "snps_list.txt"
plink --bfile genotyping_merged --extract snps_list.txt --recode A --out snp_list_plink_results
dx upload snp_list_plink_results.raw


PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to snp_list_plink_results.log.
Options in effect:
  --bfile genotyping_merged
  --extract snps_list.txt
  --out snp_list_plink_results
  --recode A

15544 MB RAM detected; reserving 7772 MB for main workspace.
129787 variants loaded from .bim file.
488377 people (223429 males, 264742 females, 206 ambiguous) loaded from .fam.
Ambiguous sex IDs written to snp_list_plink_results.nosex .
--extract: 10 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 488377 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.887258.
10 variants and 488377 people 

## Extracting SNPs from a genomic region

The code cell below allows SNPs to be exported for a defined genomic region. Change the --chr, --from-kb and --to-kb flags as required.
Your results can be found in the snp_region_plink_results.raw The file is uploaded to your project space and can be downloaded from there.

In [8]:
%%bash
plink --bfile genotyping_merged --chr 1 --from-kb 0 --to-kb 30000000 --recode A --out snp_region_plink_results
dx upload snp_region_plink_results.raw

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to snp_region_plink_results.log.
Options in effect:
  --bfile genotyping_merged
  --chr 1
  --from-kb 0
  --out snp_region_plink_results
  --recode A
  --to-kb 30000000

15544 MB RAM detected; reserving 7772 MB for main workspace.
63487 out of 129787 variants loaded from .bim file.
488377 people (223429 males, 264742 females, 206 ambiguous) loaded from .fam.
Ambiguous sex IDs written to snp_region_plink_results.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 488377 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.974083.
63487 variants and 488377 pe

## Extracting SNPs from a list of genomic regions

The code cell below allows you to filter SNPs using a file lsiting numerous genomic locations.
Prepare a file containing the required genomic locations (one per line). Rename it to genomic_regions.txt and upload to your project folder.
The file is downloaded to the instance using the dx dowload command and used to extract SNPs.
Your results can be found in the snp_region_list_plink_results.raw The file is uploaded to your project space and can be downloaded from there.

In [7]:
%%bash

dx download -f "genomic_regions.txt"

plink --bfile genotyping_merged --extract range genomic_regions.txt  --recode A --out snp_region_list_plink_results

dx upload snp_region_list_plink_results.raw


PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to snp_region_list_plink_results.log.
Options in effect:
  --bfile genotyping_merged
  --extract range genomic_regions.txt
  --out snp_region_list_plink_results
  --recode A

15544 MB RAM detected; reserving 7772 MB for main workspace.
129787 variants loaded from .bim file.
488377 people (223429 males, 264742 females, 206 ambiguous) loaded from .fam.
Ambiguous sex IDs written to snp_region_list_plink_results.nosex .
--extract range: 126452 variants excluded.
--extract range: 3335 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 488377 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929